In [29]:
from tight_binding.utilitities import compute_reciprocal_lattice_vectors_2D
from tight_binding.hamiltonians import kagome_hamiltonian_driven, kagome_hamiltonian_static, square_hamiltonian_driven, square_hamiltonian_static
from tight_binding.diagonalise import compute_time_evolution, compute_eigenstates
from tight_binding.bandstructure import compute_bandstructure2D, plot_bandstructure2D
from tight_binding.topology import compute_zak_phase, locate_dirac_strings
import numpy as np
import matplotlib.pyplot as plt

In [30]:
a_1 = np.array([1,0])
a_2 = np.array([0.5, 0.5*3**0.5])
a_3 = a_2 - a_1
r_a = a_3 / 2
r_b = a_2 / 2
r_c = a_1 / 2
offsets = np.array([r_a, r_b, r_c])

H = kagome_hamiltonian_driven(0,0,0,1,1,2,0,6,0)

In [31]:
start = np.array([0.49,0])
end = np.array([0.49,1])

zak_phase, energies = compute_zak_phase(H,a_1,a_2,offsets, start, end, 100, 6, 100)
zak_phase = np.rint(np.real(zak_phase)/np.pi) % 2
print(zak_phase)

[0. 0. 1.]


In [32]:
b_1, b_2 = compute_reciprocal_lattice_vectors_2D(a_1, a_2)
dim = 3
num_points = 100

x = np.linspace(0,1,100) # array what fraction of the path we're on
d = end - start
alpha_1 = start[0] + x * d[0]
alpha_2 = start[1] + x * d[1]
kx = alpha_1 * b_1[0] + alpha_2 * b_2[0]
ky = alpha_1 * b_1[1] + alpha_2 * b_2[1]

dk = np.array([kx[-1] - kx[0], ky[-1] - ky[0]])
diagonal = np.zeros((dim,), dtype='complex')
for i in range(dim):
    diagonal[i] = np.exp(1j*np.vdot(dk,offsets[i]))
offset_matrix = np.diag(diagonal)

In [33]:
print(ky)

[-1.77752338 -1.70423855 -1.63095373 -1.55766891 -1.48438409 -1.41109926
 -1.33781444 -1.26452962 -1.19124479 -1.11795997 -1.04467515 -0.97139033
 -0.8981055  -0.82482068 -0.75153586 -0.67825103 -0.60496621 -0.53168139
 -0.45839657 -0.38511174 -0.31182692 -0.2385421  -0.16525728 -0.09197245
 -0.01868763  0.05459719  0.12788202  0.20116684  0.27445166  0.34773648
  0.42102131  0.49430613  0.56759095  0.64087578  0.7141606   0.78744542
  0.86073024  0.93401507  1.00729989  1.08058471  1.15386953  1.22715436
  1.30043918  1.373724    1.44700883  1.52029365  1.59357847  1.66686329
  1.74014812  1.81343294  1.88671776  1.96000259  2.03328741  2.10657223
  2.17985705  2.25314188  2.3264267   2.39971152  2.47299635  2.54628117
  2.61956599  2.69285081  2.76613564  2.83942046  2.91270528  2.9859901
  3.05927493  3.13255975  3.20584457  3.2791294   3.35241422  3.42569904
  3.49898386  3.57226869  3.64555351  3.71883833  3.79212316  3.86540798
  3.9386928   4.01197762  4.08526245  4.15854727  4.

In [34]:
print(offset_matrix)

[[-1.+1.2246468e-16j  0.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j -1.+1.2246468e-16j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  1.+0.0000000e+00j]]


In [35]:
blochvectors = np.zeros((num_points,dim,dim), dtype='complex')
energies = np.zeros((num_points,dim), dtype='float')
for i in range(num_points):
    k = np.array([kx[i],ky[i]])
    eigenenergies, eigenvectors = compute_eigenstates(H, k, 6, 100)
    energies[i] = eigenenergies
    blochvectors[i] = eigenvectors

In [36]:
# Sorting the energies and blochvectors
for i in range(energies.shape[0]):
    if i == 0:
        ind = np.argsort(energies[i])
        energies[i] = energies[i,ind]
        blochvectors[i] = blochvectors[i][:,ind]
    else:
        ind = np.argsort(energies[i])
        differences = np.zeros((3,), dtype='float')
        for shift in range(3):
            ind_roll = np.roll(ind,shift)
            diff = ((energies[i,ind_roll] - energies[i-1])
                    % (2*np.pi))
            diff = (diff + 2*np.pi*np.floor((-np.pi-diff) 
                                            / (2*np.pi) + 1))
            diff = np.abs(diff)
            diff = np.sum(diff)
            differences[shift] = diff
        minimum = np.argmin(differences)
        ind = np.roll(ind, minimum)
        energies[i] = energies[i,ind]
        blochvectors[i] = blochvectors[i][:,ind]




In [37]:
print(blochvectors[0], blochvectors[-1])

[[ 0.70371595+0.j -0.0851781 +0.j -0.70529322+0.j]
 [ 0.15036811+0.j  0.98809382+0.j  0.03070842+0.j]
 [ 0.69430886+0.j -0.12766254+0.j  0.70824382+0.j]] [[ 0.70371595+0.j -0.0851781 +0.j  0.70529322+0.j]
 [ 0.15036811+0.j  0.98809382+0.j -0.03070842+0.j]
 [-0.69430886+0.j  0.12766254+0.j  0.70824382+0.j]]


In [38]:
# dealing with offsets
for i in range(blochvectors.shape[0]):
    k = np.array([kx[i],ky[i]])
    diagonal = np.array([
        np.exp(1j*np.vdot(k,a_1/2)),
        1,
        np.exp(1j*np.vdot(k,a_3/2))
    ])
    offset_matrix = np.diag(diagonal)
    blochvectors[i] = np.matmul(offset_matrix, blochvectors[i])

In [39]:
print(np.matmul(np.conjugate(np.transpose(blochvectors[0])), blochvectors[-1]))

[[ 9.99891506e-01-1.63768738e-16j -6.16972618e-07+3.00324002e-17j
  -3.14399329e-05+1.59594560e-16j]
 [-6.16972618e-07+3.03576608e-17j  9.99882434e-01-5.52943108e-18j
  -2.12782437e-06-3.03576608e-17j]
 [ 3.14399329e-05-1.70002901e-16j  2.12782437e-06+3.03576608e-17j
  -9.99990844e-01+1.67726076e-16j]]


In [40]:
# Calculating the Zak phases from the blochvectors
overlaps = np.ones((num_points - 1, dim), dtype='complex')
for i in range(num_points - 1):
    for band in range(dim):
        overlaps[i, band] = np.vdot(blochvectors[i,:,band], 
                                    blochvectors[i+1,:,band])
zak_phases = np.zeros((dim,), dtype='complex')
for band in range(dim):
    zak_phases[band] = 1j*np.log(np.prod(overlaps[:,band]))

In [41]:
zak_phases

array([ 1.61580098-0.04656461j,  3.11588888-0.05806808j,
       -1.59088625-0.04097174j])